In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<h5>The new "y" column created is our target variable. y = 1 means i enjoyed the video and y = 0 is the exact opposite.</h5>
<p>How did we create the "y" variable, you might ask. I just choose the videos i would enjoy manually and i marked the options accordingly.</p>
<p>Again, this database was not collected by my crawling code. That's because i need to improve my crawling methods after Youtube broke my previous code, again. So i will just stick with the database provided by the course i'm studying.</p>

In [2]:
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df = df[df['y'].notnull()]
df.shape

(498, 16)

In [3]:
df.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,How to Become A Machine Learning Engineer | Ho...,0.0,28.028 visualizações,Publicado em 3 de set. de 2018,Educação,#MachineLearningAlgorithms #Datasciencecourse ...,Simplilearn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,28.028 visualizações\n\n\n\n\n\n\n\n601\n\nGos...,https://i.ytimg.com/vi/-5hEYRt8JE0/maxresdefau...,1280.0,720.0,"This video on ""How to become a Machine Learnin...",1280.0,720.0,simplilearn,/channel/UCsvqVGtbbyHaMoevxPAq9Fg
1,BLOOPERS - Behind The Scenes. | DATA SCIENCE x...,0.0,1.131 visualizações,Publicado em 16 de nov. de 2018,Pessoas e blogs,#FAIL #insidezalando\n\n\n\n BLOOPERS - Beh...,Inside Zalando\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarr...,1.131 visualizações\n\n\n\n\n\n\n\n20\n\nGosto...,https://i.ytimg.com/vi/-7GiiT0yEyk/maxresdefau...,1280.0,720.0,#FAIL :) Have fun - and join our teams: https:...,1280.0,720.0,employer branding,/channel/UCTPin8TK-KRSI9zo9FoxG0g
2,Michael I. Jordan: Machine Learning: Dynamical...,1.0,1.816 visualizações,Publicado em 2 de mai. de 2019,Licença de atribuição Creative Commons (reutil...,#purdue #michaelijordan #engineering\n\n\n\n ...,Purdue Engineering\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1.816 visualizações\n\n\n\n\n\n\n\n42\n\nGosto...,https://i.ytimg.com/vi/-8yYFdV5SOc/maxresdefau...,1280.0,720.0,2019 Purdue Engineering Distinguished Lecture ...,1280.0,720.0,electrical engineer,/channel/UC8FZ6dzFVkCACLH9YoMNFog
3,Best Deep Learning Tools - Welcome.AI,0.0,1.171 visualizações,Publicado em 13 de ago. de 2019,Ciência e tecnologia,Best Deep Learning Tools - Welcome.AI,Welcome.AI\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,1.171 visualizações\n\n\n\n\n\n\n\n14\n\nGosto...,https://i.ytimg.com/vi/-9LLrwW1Vdo/maxresdefau...,1280.0,720.0,A collection of the 5 best deep learning tools...,1280.0,720.0,Watson,/channel/UC_215Y7rOAsqnFkO_hnpdIg
4,Kaggle Live-Coding: RNNs for Sarcasm Detection...,1.0,1.228 visualizações,Transmitido ao vivo em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Live-Coding: RNNs for Sarcasm Detection...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.228 visualizações\n\n\n\n\n\n\n\n28\n\nGosto...,https://i.ytimg.com/vi/-9U84J178OQ/maxresdefau...,1280.0,720.0,Join Kaggle data scientist Rachael live as she...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA


In [4]:
df.tail()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
496,Machine Learning With Python | Machine Learnin...,0.0,170.932 visualizações,Publicado em 1 de mar. de 2018,Educação,Machine Learning With Python | Machine Learnin...,Simplilearn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,170.932 visualizações\n\n\n\n\n\n\n\n2.393\n\n...,https://i.ytimg.com/vi/Q59X518JZHE/maxresdefau...,1280.0,720.0,This Machine Learning with Python tutorial giv...,1280.0,720.0,simplilearn machine learning,/channel/UCsvqVGtbbyHaMoevxPAq9Fg
497,MACHINE LEARNING and AUGMENTED REALITY,0.0,69.644 visualizações,Publicado em 31 de mai. de 2018,Ciência e tecnologia,MACHINE LEARNING and AUGMENTED REALITY,MatthewHallberg\n\n\n\n\n\n\n\n\n\n\n\n\n\nCar...,69.644 visualizações\n\n\n\n\n\n\n\n1.569\n\nG...,https://i.ytimg.com/vi/Q6ERFwQNkzo/maxresdefau...,1280.0,720.0,This tutorial is going to go through getting T...,1280.0,720.0,image classification tensorflow tutorial,/channel/UClm2DY6pj3ygKoKhEVr7KFw
498,Kaggle 대회 시작하기 #1,0.0,470 visualizações,Publicado em 10 de nov. de 2019,Pessoas e blogs,Palos Verdes\n\n\n\n Kaggle 대회 시작하기 #1,LA오빠\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando...\...,470 visualizações\n\n\n\n\n\n\n\n39\n\nGostou ...,https://i.ytimg.com/vi/Q71MI-MkGhg/maxresdefau...,1280.0,720.0,LA오남매아빠 #LA오빠 - #즐거운DataScience 오늘은 현재 진행 중인 A...,1280.0,720.0,Ashrae Energy Prediction,/channel/UCjhlZKSIRVvQxlUprmv0bdg
499,The Netacea Approach | Smarter Bot Management ...,1.0,99.855 visualizações,Publicado em 25 de abr. de 2019,Ciência e tecnologia,The Netacea Approach | Smarter Bot Management ...,Netacea\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando....,99.855 visualizações\n\n\n\n\n\n\n\n14\n\nGost...,https://i.ytimg.com/vi/QDh1vq9tVsY/hqdefault.jpg,480.0,360.0,The majority of internet traffic is now made u...,1280.0,720.0,Cyber threats,/channel/UCMLXUedkowSUnn5rSXKUzCQ
500,Stanford CS224N: NLP with Deep Learning | Wint...,0.0,17.363 visualizações,Publicado em 21 de mar. de 2019,Educação,Stanford CS224N: NLP with Deep Learning | Wint...,stanfordonline\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarr...,17.363 visualizações\n\n\n\n\n\n\n\n191\n\nGos...,https://i.ytimg.com/vi/QEw0qEa0E50/maxresdefau...,1280.0,720.0,Professor Christopher Manning & PhD Candidate ...,1280.0,720.0,NaN,/channel/UCBa5G_ESCn8Yd4vw5U-gIcg


<h1>Data Cleaning</h1>
<p>Our database collected is usually dirty. Many null fields, wrong datatypes and so on. We need to fix it with some techniques.</p>
<p>After the cleaning, we will put the changes into a new dataframe called <b>"df_limpo"</b>.</p>

In [5]:
df_limpo = pd.DataFrame(index=df.index)

<h5>Converting the column 'watch-time-text' to Datetype</h5>
<p>First of all, the field 'watch-time-text' is a string in portuguese. We want just the date with type datetime.
In order to do that, we extracted the data out of the text with a regular expression and turn it into 3 columns:
day, month, year. As seen down below.</p>

In [6]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")


In [7]:
clean_date

,0,1,2
0,3,set,2018
1,16,nov,2018
2,2,mai,2019
3,13,ago,2019
4,30,nov,2018
...,...,...,...
496,1,mar,2018
497,31,mai,2018
498,10,nov,2019
499,25,abr,2019


<h5>Cleaning the column of months</h5>
<p>The days before the tenth day of the month are built with only 1 character. In order to format the data into a more 
friendlly usage, we need to add the '0' character in front of those days. Meaning that the day 1, 2, 3 until 9 will
turn into day 01, 02, 03 until 09. I am doing this in this cell.</p>
<p>
We take every first column (clean_date[0]), whose quantity  of characters ( lenght ) of the value is equals to 1 
and turn it into a value with lenght equals to 2 characters, adding the number zero before each number below 10.</p>

In [8]:
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)


In [9]:
'''
It worked.
'''
clean_date[0]

0      03
1      16
2      02
3      13
4      30
       ..
496    01
497    31
498    10
499    25
500    21
Name: 0, Length: 498, dtype: object

<h5>Cleaning the column of months</h5>
<p>Now we need the month columns writen in English. Right now, it is writen in Portuguese. To do that, we just creating a dictionary replacing the value in Portuguese by the value in English. Then we apply those replacements to all the values in the  column, using the map function</p>

In [10]:
mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

In [11]:
clean_date[1]

0      Sep
1      Nov
2      May
3      Aug
4      Nov
      ... 
496    Mar
497    May
498    Nov
499    Apr
500    Mar
Name: 1, Length: 498, dtype: object

In [12]:
clean_date

,0,1,2
0,03,Sep,2018
1,16,Nov,2018
2,02,May,2019
3,13,Aug,2019
4,30,Nov,2018
...,...,...,...
496,01,Mar,2018
497,31,May,2018
498,10,Nov,2019
499,25,Apr,2019


<h5>Now we need a datetype</h5>
<p>We have the cleaned data into 3 columns. But we want just a single datetype column. To do that, we are going to join all the columns into just one single column separated by space. Then we are going to create a column called "date" and turn this string into a datetime type, passing the format we want as a parameter. </p>

In [13]:
clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)

df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

<h5>Cleaning the counting of views</h5>
<p>We will get only the number through the power of regular expression. Then we want to replace the dots between the numbers by an empty value. That's because we use the numbers this way in English. The fillna function exists because some viewcount has no number writen. Only a message (string). This means that when we find those undesirable strings (we want only integer numbers here), we will have the output "nan". So, we are just gonna fill those nan values with the number zero. Finally, we turn the result into a integer as desired.  </p>

In [14]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)


C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_14836/2268995852.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)


<p>Now we just need to create a column called "views" where we pass the number os views.</p>

In [15]:
df_limpo['views'] = views